# Stock Market Tracker - Multi-step Agent in a single prompt

This notebook shows how **easy** it is to build a multi-step agent using a single prompt with aisuite + MCP tools.

**What it does**: 1. Fetches latest stock market data, 2. Analyzes content and 3. Creates an HTML dashboard.

**Requirements**: `OPENAI_API_KEY` in your `.env` file

## Setup

In [1]:
import os
from datetime import datetime
from dotenv import load_dotenv
import aisuite as ai
from aisuite.mcp import MCPClient

load_dotenv()

# Initialize MCP servers for web fetching and file writing
fetch_mcp = MCPClient(command="uvx", args=["mcp-server-fetch"])
filesystem_mcp = MCPClient(
    command="npx",
    args=["-y", "@modelcontextprotocol/server-filesystem", os.getcwd()]
)

print("✓ Ready!")

✓ Ready!


## Craft instructions

In [2]:
# Define what you want the agent to do
prompt = f"""Create a stock market dashboard for {datetime.now().strftime('%Y-%m-%d')}.

**⚠️ EXECUTION RULES:**
- Execute ALL tools silently (no intermediate text responses)
- Write the HTML file FIRST, then provide summary
- If you respond with text before writing the file, the loop stops!

**TASK:**

1. **Fetch Stock Data**
   - Get top gainers from: https://finance.yahoo.com/markets/stocks/gainers/
   - Extract: ticker, company name, price, % change
   - Get 3-5 stocks

2. **Create Professional HTML Dashboard**
   - Title: "Stock Market Movers - {datetime.now().strftime('%B %d, %Y')}"
   - Display top gainers in a clean table or card layout
   - Use green color for positive % changes
   - Add professional styling:
     * Modern typography
     * Shadows and borders
     * Responsive design
     * Clean spacing
   - Include timestamp and data source

3. **Save File**
   - Use write_file to save as 'stock_dashboard.html'
   - Confirm successful write

4. **Respond**
   - ONLY after file is written, provide a brief summary
   - Include: number of stocks, top 3 gainers with % changes

Make it look like a professional financial dashboard!"""

print("✓ Agent instructions defined")

✓ Agent instructions defined


## Run Agent with MCP tools.

In [3]:
# Create agent and run it
client = ai.Client()
tools = fetch_mcp.get_callable_tools() + filesystem_mcp.get_callable_tools()

response = client.chat.completions.create(
    model="openai:gpt-4o",
    messages=[{"role": "user", "content": prompt}],
    tools=tools,
    max_turns=20
)

print("\n" + "="*60)
print("✓ DASHBOARD CREATED!")
print("="*60)
print(f"\n{response.choices[0].message.content}")


✓ DASHBOARD CREATED!

The stock market dashboard for November 11, 2025, has been successfully created and saved as 'stock_dashboard.html'. It displays the top 5 stock gainers with a professional layout. Here are the top 3 gainers:

1. **Cogent Biosciences, Inc. (COGT)** - +119.03%
2. **Navitas Semiconductor Corporation (NVTS)** - +22.45%
3. **Opendoor Technologies Inc. (OPEN)** - +21.77%

The dashboard includes modern typography, responsive design, and highlights positive percentage changes in green.


## View Dashboard

In [4]:
from IPython.display import IFrame, display

if os.path.exists('stock_dashboard.html'):
    display(IFrame(src='stock_dashboard.html', width=900, height=600))
    print("\n💡 Open 'stock_dashboard.html' in your browser for full view")
else:
    print("⚠️ Dashboard not created - check the output above")


💡 Open 'stock_dashboard.html' in your browser for full view


## Cleanup

In [ ]:
fetch_mcp.close()
filesystem_mcp.close()
print("✓ Done! That's how easy it is to build an agent with aisuite.")

---

## That's It!

In just **few lines of code**, you built an autonomous agent that:
- ✅ Fetches live stock data from the web
- ✅ Creates a beautiful HTML dashboard
- ✅ Saves it to disk

**Try it yourself**:
- Modify the prompt to fetch TOP losers and gainers.
- Modify to fetch data across multiple days.
- Modify to fetch headlines from any Finance News website to explain why stocks rose or fell. (NOTE: Many websites do not like automated calls - be mindful of their policies.)
- Try asking Agent to plot a graph with the data in the HTML page.
- Add more MCP tools (memory, search, etc.)

Remember to **increase max_turns** to a higher number if you increase the complexity of the tasks.

**Learn more**: Check out other notebooks in `examples/agents/`